In [1]:
!pip install ipywidgets

In [3]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 1.9 MB/s eta 0:00:0000:0100:01


In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import os
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join('.', 'db.sqlite3'),
    }
}
import json


import chromadb

import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS
import os

config_list = [
    {"model": "gpt-3.5-turbo-0125", "api_key": os.getenv('OPENAI_API_KEYS'), "api_type": "openai"},
]

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-3.5-turbo-0125']


In [5]:
autogen

<module 'autogen' from '/workspaces/Contract-Advisor-RAG/venv/lib/python3.10/site-packages/autogen/__init__.py'>

In [3]:
from collections import defaultdict
import datetime
timer_array = []

In [7]:
from autogen import AssistantAgent, UserProxyAgent

In [4]:
worker = autogen.AssistantAgent(
    name="worker",
    llm_config={"config_list": config_list, "cache_seed":42},
    code_execution_config=False,

    # the default system message of the AssistantAgent is overwritten here
    system_message="You are a helpful AI worker. You are a worker agent who does exactly what you are told to do.",
)


In [5]:
import sys
sys.path.append("../")

In [6]:
from scripts.retrieval.retriever import retriever

In [7]:
retriever_obj = retriever()

def ask_worker(message):
     # context = ""
      #if retrival == "TRUE":
       # try:
      print("retrival started")
      relevant_docs = retriever_obj.invoke(message)
      context = "\n " + "Answer the question(message) from this important information to achieve the tasks: " + vars(relevant_docs[0])['page_content']# + " " + vars(relevant_docs[1])['page_content']
      #context = "\n " + "Answer the question(message) from this important information to achieve the tasks: " + str(retriever_obj.invoke(message))
      
      print(context)
      print("retrival ended")

      worker.llm_config = {"config_list": config_list, "response_format":{ "type": "json_object" }}
      
      prepared_message = [{'content':  message + context ,  'role': 'assistant'}]

      print(" ************** Fully Prepared Worker Context *************************")
      print(prepared_message[0]["content"])
      print(" ************** Fully Prepared Worker Context *************************")

      worker.llm_config = {"config_list": config_list, "response_format":{ "type": "json_object" }}
      result = worker.generate_reply(messages=prepared_message)

      return result
        #except:
         # print("retrival failed")
          # context = ""

/workspaces/Contract-Advisor-RAG/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
message = 'What are the payments to the Advisor under the Agreement?'
#result = retriever_obj.invoke(message)

In [11]:
ret = ask_worker(message)

retrival started



 Answer the question(message) from this important information to achieve the tasks: Termination : Either party , at any given time, may terminate this Agreement, for any reason  whatsoever , with or without cause, upon fourteen (14) days’ prior written notice. Notwithstanding the  above, the Company may terminate this Agreement immediately and without prior notice if Advisor  refuses or is unable to perform the Services, or is in breach of any provision of this Agreement. 6. Compensation: 6.1 As full and sole consideration for the Services, and subject to the performance of the Services,  the Company shall pay the Advisor , against an invoice provided to the Company by Advisor ,  hourly fees at a rate of USD 9 (nine) per Billable Hour as defined below , limited to a maximum  of USD 1,500 per month (the " Fees "). In addition, the Company shall pay the advisor USD 100  per month to finance a workspace for the Advisor , as long as the Advisor actually hires a  professional workspace (th

In [12]:
ret

'The Advisor will be compensated with hourly fees at a rate of USD 9 per Billable Hour, limited to a maximum of USD 1,500 per month. In addition, the company will pay the Advisor USD 100 per month for a workspace expense, provided that the Advisor actually hires a professional workspace. The Fees and Workspace Expense for each month are payable by the company no later than ten days from the beginning of the following month, upon receipt of a duly issued tax invoice. The Advisor will not be entitled to any additional fees or expense reimbursement, except as specified in the Agreement.'

In [39]:
ret

'The payments to the Advisor under the Agreement are as follows:\n\n1. The Company shall pay the Advisor hourly fees at a rate of USD 9 per Billable Hour, limited to a maximum of USD 1,500 per month.\n\n2. The Company shall also pay the Advisor USD 100 per month to finance a workspace, provided that the Advisor actually hires a professional workspace.\n\n3. The Fees and the Workspace Expense for each month shall be payable by no later than ten (10) days from the beginning of the following month, against receipt by the Company of a duly issued tax invoice.\n\nAdvisor will not be entitled to any additional fees or expense reimbursement except as expressly provided for in the Agreement.'

In [29]:
retriever_obj.get_relevant_documents(message)

/workspaces/Contract-Advisor-RAG/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='The parties consent to the  exclusive jurisdiction and venue of Tel Aviv courts for any lawsuit filed arising from or relating to  this Agreement. 11. Notices : Notices under this Agreement shall be delivered to the party’ s email address as follows:  Company: info@cloudcorp.com , Advisor: jackrobinson@gmail.com , or in any the other means with a  proof of acceptance by the other party . IN WITNESS WHEREOF the parties have executed this Agreement as of the date first above written. Cloud Investments Ltd. Advisor By: ________________________ By:________________________ Name: Silvan Joseph Name: Jack Robinson Title: CEO  - 4 - Confidentiality, None Compete and IP Ownership Undertaking Appendix A to Advisory Service Agreement as of June 15 th , 2023 THIS CONFIDENTIALITY UNDER TAKING (“ Undertaking ”) is entered into as of June 15 th , 2023 (“ Effective Date ”), by Mr. Jack  Robinson, Passport Number 780055578, residing at 1 Rabin st, Tel Aviv, Israel, Email: jackro

In [38]:
vars(result[0])['page_content'] + vars(result[0])['page_content']

'The parties consent to the  exclusive jurisdiction and venue of Tel Aviv courts for any lawsuit filed arising from or relating to  this Agreement. 11. Notices : Notices under this Agreement shall be delivered to the party’ s email address as follows:  Company: info@cloudcorp.com , Advisor: jackrobinson@gmail.com , or in any the other means with a  proof of acceptance by the other party . IN WITNESS WHEREOF the parties have executed this Agreement as of the date first above written. Cloud Investments Ltd. Advisor By: ________________________ By:________________________ Name: Silvan Joseph Name: Jack Robinson Title: CEO  - 4 - Confidentiality, None Compete and IP Ownership Undertaking Appendix A to Advisory Service Agreement as of June 15 th , 2023 THIS CONFIDENTIALITY UNDER TAKING (“ Undertaking ”) is entered into as of June 15 th , 2023 (“ Effective Date ”), by Mr. Jack  Robinson, Passport Number 780055578, residing at 1 Rabin st, Tel Aviv, Israel, Email: jackrobinson@gmail.com, (“ Ad

In [42]:
str(result[0]) 

"[Document(page_content='The parties consent to the  exclusive jurisdiction and venue of Tel Aviv courts for any lawsuit filed arising from or relating to  this Agreement. 11. Notices : Notices under this Agreement shall be delivered to the party’ s email address as follows:  Company: info@cloudcorp.com , Advisor: jackrobinson@gmail.com , or in any the other means with a  proof of acceptance by the other party . IN WITNESS WHEREOF the parties have executed this Agreement as of the date first above written. Cloud Investments Ltd. Advisor By: ________________________ By:________________________ Name: Silvan Joseph Name: Jack Robinson Title: CEO  - 4 - Confidentiality, None Compete and IP Ownership Undertaking Appendix A to Advisory Service Agreement as of June 15 th , 2023 THIS CONFIDENTIALITY UNDER TAKING (“ Undertaking ”) is entered into as of June 15 th , 2023 (“ Effective Date ”), by Mr. Jack  Robinson, Passport Number 780055578, residing at 1 Rabin st, Tel Aviv, Israel, Email: jackr

In [41]:
result

[Document(page_content='The parties consent to the  exclusive jurisdiction and venue of Tel Aviv courts for any lawsuit filed arising from or relating to  this Agreement. 11. Notices : Notices under this Agreement shall be delivered to the party’ s email address as follows:  Company: info@cloudcorp.com , Advisor: jackrobinson@gmail.com , or in any the other means with a  proof of acceptance by the other party . IN WITNESS WHEREOF the parties have executed this Agreement as of the date first above written. Cloud Investments Ltd. Advisor By: ________________________ By:________________________ Name: Silvan Joseph Name: Jack Robinson Title: CEO  - 4 - Confidentiality, None Compete and IP Ownership Undertaking Appendix A to Advisory Service Agreement as of June 15 th , 2023 THIS CONFIDENTIALITY UNDER TAKING (“ Undertaking ”) is entered into as of June 15 th , 2023 (“ Effective Date ”), by Mr. Jack  Robinson, Passport Number 780055578, residing at 1 Rabin st, Tel Aviv, Israel, Email: jackro

In [25]:
assistant = autogen.AssistantAgent(
    name="assistant",
    # system_message="""
    
    # Your goal is to assist users by providing precise and helpful answers based on a contract document.
    # The document in question is an Advisory Services Agreement.
    # Given the user query, generate a well-structured and informative response by extracting relevant information from the document.
    # Ensure the response is concise, accurate, and directly addresses the query.
    # """,

    system_message = """
    You are an AI expert specialized in contract analysis and advisory services. Your primary objective is to plan steps to appropriately solve users'\
     requests or tasks using worker function calling.

    The main problems you will be solving include:
    - Answering contract questions

    Your tasks:
    1. Use the `ask_worker` function to delegate each step, ensuring it returns the results upon task completion.
    2. VERY IMPORTANT: Before you return any result ensure that you do not need worker skills to perform a task.

    Key guidelines:
    - ALWAYS use the ask_worker function for the task and send it only the most important instructions and commands.
    - FORCE the ask_worker function to only give you the answer for your request without adding explanation or guidance what it achieved.

    Communication:
    - Your answer should be constructed in the given language the user used to ask.
    - Before responding to the user make sure you used all the skills the worker function provides.
    - When communicating with the Worker make sure to send The most Important context to it.

    """,

    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
        "functions": [
             {
                        "name": "ask_worker",
                        "description": "ask worker to:  1. generate questions for requirement gathering, 2. retrieve relevant contract based on the topic sent as message, 3. Return the answer based on the message and the retrived relevant document",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "message": {
                                    "type": "string",
                                    "description": "question to ask worker. Make sure the question include enough context, such as the required task. The worker does not know the conversation between you and the user, unless you share the conversation with the worker. Ensure the original question of the user is also passed as context such that the worker is generating intermediate steps necessary to address it. It is VERY IMPORTANT that you provide a clear and autoritative feedback to the worker to stay focused on answering the original question."
                                    },
                                # "message_type": {
                                #     "type": "string",
                                #     "description": "Type of work you want it to achieve. Options are: 'REQUIREMENT GATHERING','SIMPLE QUESTION ANSWERING'"
                                # },
                                # "retrival":{
                                #     "type": "string",
                                #     "description": "Becomes 'TRUE' or 'FALSE' which would indicate to perform retrieval of relevant articles from the law to achieve the task"
                                # },
                            },
                                
                        }
                        }
             
        ],
    },
)

In [46]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    #is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    # code_execution_config={"work_dir": "planning"},
    function_map={"ask_worker": ask_worker },
)


In [27]:
print("Actual usage summary for assistant (excluding completion from cache):", assistant.get_actual_usage())

Actual usage summary for assistant (excluding completion from cache): {'total_cost': 0.0013050000000000002, 'gpt-3.5-turbo-0125': {'cost': 0.0013050000000000002, 'prompt_tokens': 2238, 'completion_tokens': 124, 'total_tokens': 2362}}


In [22]:
def chat_with_agent(message):
  # combined_message, lang = translate_to_dest(message,"en")
  user_proxy.initiate_chat(
      assistant,
      #clear_history=False,
     # silent=True,
      message=message
  )

In [50]:
message = 'What happens if the Advisor claims compensation based on employment relationship with the Company? '

In [51]:
chat_with_agent(message)

user_proxy (to assistant):

What happens if the Advisor claims compensation based on employment relationship with the Company? 

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function call: ask_worker *****
Arguments: 
{"message":"Explain the consequences if the Advisor claims compensation based on an employment relationship with the Company."}
***********************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION ask_worker...
retrival started

 Answer the question(message) from this important information to achieve the tasks: Without derogating from the above, if it is adjudicated or otherwise determined by any  governmental authority that the Advisor and/or anyone on Advisor's behalf, is, regardless of the terms  of this Agreement, an employee of the Company , then payments to Advisor hereunder shall be  reduced effect

In [52]:
[messages for agent, messages in user_proxy.chat_messages.items()][0][-1]['content']

'If the Advisor claims compensation based on an employment relationship with the Company, 60% of the payments will be considered as salary payments, and 40% for other statutory rights and benefits. The Company can offset amounts owed to the Advisor from other payments due, and the Advisor must indemnify the Company for any losses in case of an employer/employee relationship. '

In [83]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
)
from ragas import evaluate
from scripts.evaluation.evaluation_result import visualize_result

In [59]:
import docx
def load_evaluation_dataset(docx_path):
    """
    Loads an evaluation dataset from a DOCX file.

    Parameters:
    docx_path (str): Path to the DOCX file containing the questions and answers.

    Returns:
    tuple: Two lists, one with questions and one with corresponding answers.
    """
    doc = docx.Document(docx_path)
    questions = []
    answers = []

    for para in doc.paragraphs:
        text = para.text.strip()
        if text.startswith("Q"):
            questions.append(text)
        elif text.startswith("A"):
            answers.append(text)

    return questions, answers

In [61]:
questions, ground_truth = load_evaluation_dataset("../data/Robinson Q&A.docx")

In [71]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_key = os.getenv("OPENAI_API_KEYS")
pinecone_key = os.getenv("PINECONE_API_KEYS")

os.environ["OPENAI_API_KEY"] = openai_key

In [85]:
from datasets import Dataset
import asyncio
def evaluate_metrics():
    """
    Evaluates performance metrics for generated responses.

    Returns:
    dict: A dictionary with the evaluation metrics.
    """
    answers = []
    contexts = []
    loop = asyncio.get_event_loop()

    for question in questions:
        #relevant_docs = retriever_obj.invoke(question)
    #    context = "\n " + "Answer the question(message) from this important information to achieve the tasks: " + vars(relevant_docs[0])['page_content']# + " " + vars(relevant_docs[1])['page_content']
   
        result = ask_worker(question)
        answers.append(result)
        contexts.append(
            #vars(relevant_docs[0])['page_content']
            [docs.page_content for docs in retriever_obj.get_relevant_documents(question)]
        )

    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truth,
    }

    dataset = Dataset.from_dict(data)

    print(dataset)

    result = evaluate(
        dataset=dataset,
        metrics=[
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy,
            answer_similarity,
        ],
    )

    return result

In [86]:
retriever_obj = retriever()

In [87]:
import asyncio
result = asyncio.run(evaluate_metrics())

print(result)

retrival started



 Answer the question(message) from this important information to achieve the tasks: The parties consent to the  exclusive jurisdiction and venue of Tel Aviv courts for any lawsuit filed arising from or relating to  this Agreement. 11. Notices : Notices under this Agreement shall be delivered to the party’ s email address as follows:  Company: info@cloudcorp.com , Advisor: jackrobinson@gmail.com , or in any the other means with a  proof of acceptance by the other party . IN WITNESS WHEREOF the parties have executed this Agreement as of the date first above written. Cloud Investments Ltd. Advisor By: ________________________ By:________________________ Name: Silvan Joseph Name: Jack Robinson Title: CEO  - 4 - Confidentiality, None Compete and IP Ownership Undertaking Appendix A to Advisory Service Agreement as of June 15 th , 2023 THIS CONFIDENTIALITY UNDER TAKING (“ Undertaking ”) is entered into as of June 15 th , 2023 (“ Effective Date ”), by Mr. Jack  Robinson, Passport Number 78005

Exception in thread Thread-81:
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/workspaces/Contract-Advisor-RAG/venv/lib/python3.10/site-packages/ragas/executor.py", line 87, in run
    results = self.loop.run_until_complete(self._aresults())
  File "/home/codespace/.python/current/lib/python3.10/asyncio/base_events.py", line 625, in run_until_complete
    self._check_running()
  File "/home/codespace/.python/current/lib/python3.10/asyncio/base_events.py", line 584, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running


Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 10
})


/home/codespace/.python/current/lib/python3.10/threading.py:1018: RuntimeWarning: coroutine 'Runner._aresults' was never awaited
  self._invoke_excepthook(self)


ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

/workspaces/Contract-Advisor-RAG/venv/lib/python3.10/site-packages/ipykernel/iostream.py:123: RuntimeWarning: coroutine 'as_completed.<locals>.sema_coro' was never awaited
  await self._event_pipe_gc()


In [84]:
from datasets import Dataset
import asyncio
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
)
from ragas import evaluate

async def evaluate_metrics_async():
    """
    Evaluates performance metrics for generated responses asynchronously.

    Returns:
    dict: A dictionary with the evaluation metrics.
    """
    answers = []
    contexts = []

    for question in questions:
        # Retrieve relevant documents (dummy code for retriever_obj)
        relevant_docs = retriever_obj.invoke(question)
        context = "\n" + "Answer the question(message) from this important information to achieve the tasks: " + relevant_docs[0]['page_content']
        
        result = ask_worker(question)  # Replace with your actual function to generate answers
        answers.append(result)
        contexts.append([doc['page_content'] for doc in relevant_docs])

    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truth,
    }

    dataset = Dataset.from_dict(data)

    result = await evaluate(
        dataset=dataset,
        metrics=[
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy,
            answer_similarity,
        ],
    )

    return result

def evaluate_metrics():
    """
    Wrapper function to run the asynchronous evaluation.

    Returns:
    dict: A dictionary with the evaluation metrics.
    """
    return asyncio.run(evaluate_metrics_async())

# Assuming questions and ground_truth are defined
# questions = [
#     "Who are the parties to the Agreement and what are their defined names?",
#     "What is the termination notice?",
#     # Add more questions as needed
# ]

# ground_truth = [
#     "The parties are Cloud Investments Ltd. and Jack Robinson.",
#     "Either party may terminate the Agreement with 14 days' prior written notice.",
#     # Add corresponding ground truth answers
# ]

# Assuming retriever_obj and ask_worker are defined and initialized
# Initialize your retriever_obj and ask_worker here

# Run the evaluation
result = evaluate_metrics()

print(result)


RuntimeError: asyncio.run() cannot be called from a running event loop

In [46]:
# !pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize the list to hold messages
messages = []


def handle_input(sender):
    global messages
    user_input = sender.value
    messages.append(f"User: {user_input}")
    chat_with_agent(user_input)
    messages.append("")
    display_messages()

# Function to display messages
def display_messages():

    text_input = widgets.Text(
        value='',
        placeholder='Type your message',
        description='Chat:',
        disabled=False
    )
    text_input.on_submit(handle_input)
    display(text_input)


#display_messages()

In [47]:
display_messages()

/tmp/ipykernel_6594/4125127347.py:26: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text_input.on_submit(handle_input)


Text(value='', description='Chat:', placeholder='Type your message')

user_proxy (to assistant):

Who are the parties to the Agreement and what are their defined names?

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function call: ask_worker *****
Arguments: 
{"message":"Identify the parties to the Agreement and provide their defined names."}
***********************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION ask_worker...
retrival started

 Answer the question(message) from this important information to achieve the tasks: IN WITNESS WHEREOF , the undersigned has executed this Undertaking as of the Effective Date. Advisor By: _____________________________________ Name: Jack Robinson.
retrival ended
 ************** Fully Prepared Worker Context *************************
Identify the parties to the Agreement and provide their defined names.
 Answer the question(message) from this important

/tmp/ipykernel_6594/4125127347.py:26: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text_input.on_submit(handle_input)


Text(value='', description='Chat:', placeholder='Type your message')